In [0]:
storageAccountName = 'equitymarketstorage'
storageAccountAccessKey = '32Q+pSDygN/vBS/+2OPfjXJrW9+T4gogsOJY3mEob/GON/Hls27FX1ZaxbBxEjsmA+/+XJn1AXkFj2dUSrOeWA=='
blobContainerName = 'firstcontainer'


if not any(mount.mountPoint == '/mnt/FileStore/MountFolder/' for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
    source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
    mount_point = "/mnt/FileStore/MountFolder/",
    extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
  )
  except Exception as e:
    print("already mounted. Try to unmount first")
    

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType ,LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DateType, TimestampType, DecimalType
import datetime


In [0]:
df=spark.read.parquet("dbfs:/mnt/FileStore/MountFolder/data/data_csv.parquet/partition=T/trade_dt={}".format('2020-08-06'))
df.createOrReplaceTempView("trades")

In [0]:
df = spark.sql("select symbol, exchange, event_tm, event_seq_nb, trade_pr from trades")
df.createOrReplaceTempView("tmp_trade_moving_avg")

In [0]:
mov_avg_df = spark.sql("""
select symbol, exchange, event_tm, event_seq_nb, trade_pr,
AVG(trade_pr) OVER(PARTITION BY (symbol) ORDER BY CAST(event_tm as timestamp)
RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW) as mov_avg_pr
from tmp_trade_moving_avg
""")

In [0]:
mov_avg_df.write.mode("overwrite").saveAsTable("temp_trade_moving_avg")

In [0]:
import datetime
date = datetime.datetime.strptime('2020-08-06', '%Y-%m-%d')
prev_date =date - datetime.timedelta(days=1)
#next_date_str = datetime.datetime.strptime('2020-08-06', '%Y-%m-%d')
#next_date_str = next_date_str.replace('00:00:00','')

In [0]:
prev_day_df=spark.read.parquet("dbfs:/mnt/FileStore/MountFolder/data/data_csv.parquet/partition=T/trade_dt={}".format("2020-08-05"))
prev_day_df.createOrReplaceTempView("tmp_last_trade")

In [0]:
last_pr_df = spark.sql("""select symbol, exchange, last_pr from (select
symbol, exchange, event_tm, event_seq_nb, trade_pr, AVG(trade_pr) OVER(PARTITION BY (symbol) ORDER BY CAST(event_tm as timestamp)
RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW) AS last_pr
FROM tmp_trade_moving_avg)""")


In [0]:
spark.sql("DROP TABLE IF EXISTS temp_last_trade")
last_pr_df.write.saveAsTable("temp_last_trade")

In [0]:
quotes = spark.read.parquet('dbfs:/mnt/FileStore/MountFolder/data/data_csv.parquet/partition=Q')
quotes.createOrReplaceTempView("quotes")

In [0]:
quote_union = spark.sql("""
SELECT rec_type,symbol,event_tm,event_seq_nb,exchange,bid_pr,bid_size,ask_pr,ask_size,null as trade_pr,null as mov_avg_pr FROM quotes
UNION
SELECT "T" as rec_type,symbol,event_tm,event_seq_nb,exchange,null as bid_pr,null as bid_size,null as ask_pr,null as ask_size,trade_pr,mov_avg_pr FROM temp_trade_moving_avg
""")

In [0]:
quote_union.createOrReplaceTempView("quote_union")

In [0]:
quote_union_update = spark.sql("""
select *,
last_value(trade_pr,true) OVER(PARTITION BY (symbol) 
ORDER BY CAST(event_tm AS timestamp) DESC) as last_trade_pr,
last_value(mov_avg_pr,true) OVER(PARTITION BY (symbol) 
ORDER BY CAST(event_tm AS timestamp) DESC) as last_mov_avg_pr
from quote_union
""")

In [0]:
quote_union_update.createOrReplaceTempView("quote_union_update")

In [0]:
quote_update = spark.sql("""
select symbol, event_tm, event_seq_nb, exchange, bid_pr, bid_size, ask_pr, ask_size, last_trade_pr, last_mov_avg_pr from quote_union_update 
where rec_type = 'Q'
""")
quote_update.createOrReplaceTempView("quote_update")

In [0]:
quote_final=spark.sql("""
select A.symbol, event_tm, event_seq_nb, exchange, bid_pr, bid_size, ask_pr, ask_size, last_trade_pr, last_mov_avg_pr, bid_pr-close_pr as bid_pr_mov, ask_pr-close_pr as ask_pr_mv
from quote_update A LEFT OUTER JOIN (select distinct symbol,last_pr as close_pr from temp_last_trade) B on A.symbol = B.symbol
""")

In [0]:
quote_update.write.mode("overwrite").parquet("dbfs:/mnt/FileStore/MountFolder/data/data_csv.parquet/quote-trade-analytical/date={}".format("2020-08-06"))